# Nhiệm vụ phân loại văn bản

Như đã đề cập, chúng ta sẽ tập trung vào nhiệm vụ phân loại văn bản đơn giản dựa trên tập dữ liệu **AG_NEWS**, đó là phân loại tiêu đề tin tức vào một trong 4 danh mục: Thế giới, Thể thao, Kinh doanh và Khoa học/Công nghệ.

## Tập dữ liệu

Tập dữ liệu này được tích hợp trong mô-đun [`torchtext`](https://github.com/pytorch/text), vì vậy chúng ta có thể dễ dàng truy cập nó.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Ở đây, `train_dataset` và `test_dataset` chứa các tập hợp trả về các cặp gồm nhãn (số của lớp) và văn bản tương ứng, ví dụ:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Vậy, hãy in ra 10 tiêu đề mới đầu tiên từ tập dữ liệu của chúng ta:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Bởi vì các tập dữ liệu là các trình lặp, nếu chúng ta muốn sử dụng dữ liệu nhiều lần, chúng ta cần chuyển nó thành danh sách:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Phân tách từ

Bây giờ chúng ta cần chuyển đổi văn bản thành **số** có thể được biểu diễn dưới dạng tensor. Nếu chúng ta muốn biểu diễn ở cấp độ từ, cần thực hiện hai việc:
* sử dụng **tokenizer** để chia văn bản thành **token**
* xây dựng một **từ vựng** cho các token đó.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Sử dụng từ vựng, chúng ta có thể dễ dàng mã hóa chuỗi đã được phân tách thành một tập hợp các số:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Biểu diễn văn bản bằng Bag of Words

Vì từ ngữ mang ý nghĩa, đôi khi chúng ta có thể hiểu được nội dung của một văn bản chỉ bằng cách nhìn vào các từ riêng lẻ, bất kể thứ tự của chúng trong câu. Ví dụ, khi phân loại tin tức, các từ như *thời tiết*, *tuyết* có khả năng chỉ ra *dự báo thời tiết*, trong khi các từ như *cổ phiếu*, *đô la* sẽ liên quan đến *tin tức tài chính*.

**Bag of Words** (BoW) là cách biểu diễn vector truyền thống được sử dụng phổ biến nhất. Mỗi từ được liên kết với một chỉ số vector, phần tử trong vector chứa số lần xuất hiện của từ đó trong một tài liệu cụ thể.

![Hình ảnh minh họa cách biểu diễn vector Bag of Words trong bộ nhớ.](../../../../../translated_images/vi/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Bạn cũng có thể nghĩ về BoW như tổng của tất cả các vector mã hóa một-hot cho từng từ riêng lẻ trong văn bản.

Dưới đây là một ví dụ về cách tạo biểu diễn Bag of Words bằng thư viện Scikit Learn trong Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Để tính toán vector bag-of-words từ biểu diễn vector của tập dữ liệu AG_NEWS, chúng ta có thể sử dụng hàm sau:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Lưu ý:** Ở đây chúng ta sử dụng biến toàn cục `vocab_size` để chỉ định kích thước mặc định của từ vựng. Vì kích thước từ vựng thường khá lớn, chúng ta có thể giới hạn kích thước từ vựng bằng cách chỉ sử dụng các từ phổ biến nhất. Hãy thử giảm giá trị của `vocab_size` và chạy đoạn mã dưới đây, và xem nó ảnh hưởng như thế nào đến độ chính xác. Bạn nên mong đợi một số giảm độ chính xác, nhưng không đáng kể, để đổi lấy hiệu suất cao hơn.


## Huấn luyện bộ phân loại BoW

Bây giờ khi chúng ta đã học cách xây dựng biểu diễn Bag-of-Words cho văn bản, hãy huấn luyện một bộ phân loại dựa trên đó. Đầu tiên, chúng ta cần chuyển đổi tập dữ liệu để huấn luyện sao cho tất cả các biểu diễn vector vị trí được chuyển đổi thành biểu diễn bag-of-words. Điều này có thể thực hiện bằng cách truyền hàm `bowify` như tham số `collate_fn` vào `DataLoader` tiêu chuẩn của torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Bây giờ hãy định nghĩa một mạng nơ-ron phân loại đơn giản chứa một lớp tuyến tính. Kích thước của vector đầu vào bằng `vocab_size`, và kích thước đầu ra tương ứng với số lượng lớp (4). Vì chúng ta đang giải quyết bài toán phân loại, hàm kích hoạt cuối cùng là `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Bây giờ chúng ta sẽ định nghĩa vòng lặp huấn luyện tiêu chuẩn của PyTorch. Vì tập dữ liệu của chúng ta khá lớn, cho mục đích giảng dạy, chúng ta sẽ chỉ huấn luyện trong một epoch, và đôi khi thậm chí ít hơn một epoch (việc chỉ định tham số `epoch_size` cho phép chúng ta giới hạn huấn luyện). Chúng ta cũng sẽ báo cáo độ chính xác huấn luyện tích lũy trong quá trình huấn luyện; tần suất báo cáo được chỉ định bằng tham số `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams và N-Grams

Một hạn chế của phương pháp bag of words là một số từ thuộc về các cụm từ nhiều từ, ví dụ, từ 'hot dog' có ý nghĩa hoàn toàn khác so với các từ 'hot' và 'dog' trong các ngữ cảnh khác. Nếu chúng ta luôn biểu diễn các từ 'hot' và 'dog' bằng cùng một vector, điều này có thể gây nhầm lẫn cho mô hình của chúng ta.

Để giải quyết vấn đề này, **biểu diễn N-gram** thường được sử dụng trong các phương pháp phân loại tài liệu, nơi tần suất của mỗi từ, cụm hai từ hoặc cụm ba từ là một đặc điểm hữu ích để huấn luyện các bộ phân loại. Trong biểu diễn bigram, ví dụ, chúng ta sẽ thêm tất cả các cặp từ vào từ vựng, bên cạnh các từ gốc.

Dưới đây là một ví dụ về cách tạo biểu diễn bag of words bigram bằng Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Nhược điểm chính của phương pháp N-gram là kích thước từ vựng bắt đầu tăng rất nhanh. Trong thực tế, chúng ta cần kết hợp biểu diễn N-gram với một số kỹ thuật giảm chiều, chẳng hạn như *embeddings*, mà chúng ta sẽ thảo luận trong bài học tiếp theo.

Để sử dụng biểu diễn N-gram trong tập dữ liệu **AG News**, chúng ta cần xây dựng từ vựng ngram đặc biệt:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Chúng ta có thể sử dụng cùng đoạn mã như trên để huấn luyện bộ phân loại, tuy nhiên, cách này sẽ rất tốn bộ nhớ. Trong phần tiếp theo, chúng ta sẽ huấn luyện bộ phân loại bigram bằng cách sử dụng embeddings.

> **Lưu ý:** Bạn chỉ nên giữ lại những ngram xuất hiện trong văn bản nhiều hơn số lần được chỉ định. Điều này sẽ đảm bảo rằng các bigram hiếm gặp sẽ bị loại bỏ, và giảm đáng kể kích thước không gian chiều. Để làm điều này, hãy đặt tham số `min_freq` ở giá trị cao hơn, và quan sát sự thay đổi độ dài của từ vựng.


## Tần suất thuật ngữ - Tần suất nghịch tài liệu TF-IDF

Trong biểu diễn BoW, số lần xuất hiện của từ được đánh giá ngang nhau, bất kể bản thân từ đó là gì. Tuy nhiên, rõ ràng rằng các từ xuất hiện thường xuyên, như *a*, *in*, v.v., ít quan trọng hơn đối với việc phân loại so với các thuật ngữ chuyên biệt. Thực tế, trong hầu hết các nhiệm vụ NLP, một số từ có mức độ liên quan cao hơn các từ khác.

**TF-IDF** là viết tắt của **tần suất thuật ngữ–tần suất nghịch tài liệu**. Đây là một biến thể của mô hình bag of words, trong đó thay vì giá trị nhị phân 0/1 biểu thị sự xuất hiện của một từ trong tài liệu, một giá trị số thực được sử dụng, liên quan đến tần suất xuất hiện của từ trong tập dữ liệu.

Một cách chính xác hơn, trọng số $w_{ij}$ của một từ $i$ trong tài liệu $j$ được định nghĩa như sau:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
trong đó:
* $tf_{ij}$ là số lần xuất hiện của $i$ trong $j$, tức là giá trị BoW mà chúng ta đã thấy trước đó
* $N$ là số lượng tài liệu trong tập hợp
* $df_i$ là số lượng tài liệu chứa từ $i$ trong toàn bộ tập hợp

Giá trị TF-IDF $w_{ij}$ tăng tỷ lệ thuận với số lần một từ xuất hiện trong tài liệu và được điều chỉnh bởi số lượng tài liệu trong tập hợp chứa từ đó, giúp cân bằng thực tế rằng một số từ xuất hiện thường xuyên hơn các từ khác. Ví dụ, nếu từ xuất hiện trong *mọi* tài liệu trong tập hợp, $df_i=N$, và $w_{ij}=0$, những thuật ngữ đó sẽ hoàn toàn bị bỏ qua.

Bạn có thể dễ dàng tạo vector hóa TF-IDF cho văn bản bằng cách sử dụng Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Kết luận

Tuy nhiên, mặc dù các biểu diễn TF-IDF cung cấp trọng số tần suất cho các từ khác nhau, chúng không thể biểu diễn ý nghĩa hoặc thứ tự. Như nhà ngôn ngữ học nổi tiếng J. R. Firth đã nói vào năm 1935: “Ý nghĩa đầy đủ của một từ luôn mang tính ngữ cảnh, và không thể nghiên cứu ý nghĩa mà không có ngữ cảnh một cách nghiêm túc.”. Sau này trong khóa học, chúng ta sẽ học cách nắm bắt thông tin ngữ cảnh từ văn bản bằng cách sử dụng mô hình ngôn ngữ.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
